In [2]:
# 数据分析/处理
import numpy as np
import pandas as pd

# 搭建神经网络
import torch
import torch.nn as nn
import torch.nn.functional as function
import torch.nn.init as init
from torch import optim
from torch.utils.data import Dataset,DataLoader

# 数据可视化
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

d:\Anaconda\envs\DL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 宇宙的答案
np.random.seed(42)
torch.manual_seed(42)

### 关于词嵌入（Word Embedding）

词嵌入（Word Embedding）指的是根据词汇在文本中的上下文将词汇转化为稠密化的向量（也称为分布式表示）表示的一类算法。之所以称这种技术为嵌入（Embedding）是由于它可以将词汇表示在一个向量空间中，我们甚至可以使用这些表示（一般简称为词向量）进行一些带有语义的运算（例如：king-man+woman=queen，或计算余弦相似度）。最早的基于DL的词嵌入技术Word2Vec由传奇捷克NLP研究员Tomas Mikolov于2013年前后与Ilya、Jeff Dean等人于Google共同开发。

Word2Vec的基本假设就是：每个词汇的含义，取决于它**可能**出现的上下文（事后证明这个假设基本上是正确的）。Word2Vec突破了之前的各种词汇表示方法无法联系上下文的缺点,可以显示词之间的相似关系，且稠密的向量化的表示更适于计算和存储。但是Word2Vec仍然有一些缺点，例如由于反义词由于出现的语境比较接近，所以两个反义词在向量空间中比较接近，正常来讲一对反义词的词向量应该成一个平角才对。

GloVe则是斯坦福大学提出的针对Word2Vec的改进，相较于Word2Vec改进了面对生僻词时的等情况下的性能（尽管Mikolov仍然认为GloVe的效果逊于Word2Vec）。

根据原项目的要求，应当使用以下三种方式对词汇进行表示：
 + Word2Vec（对应原要求中的word embedding，这样处理是因为Word2Vec影响力太大，所以有时有人会用word embedding指代它）
 + 随机初始化
 + GloVe

然后搭建RNN和CNN进行分类。

欲了解更多可以参考以下资料：

[Word2Vec的介绍1](https://zhuanlan.zhihu.com/p/61635013)

[Word2Vec的介绍2](https://zhuanlan.zhihu.com/p/26306795)

[GloVe的介绍1](https://zhuanlan.zhihu.com/p/50946044)

[GloVe的介绍2](https://zhuanlan.zhihu.com/p/42073620)

[GloVe的介绍3](https://zh.gluon.ai/chapter_natural-language-processing/glove.html)

另外，Word2Vec和GloVe并不是唯一的一种分布式词向量表示方法。另一类常用的分布式词表示方法还有ELMo和Bert，这类方法可以通过考察词语所在的上下文语境动态的形成词向量。

### 载入数据，分析同task1

In [13]:
# 数据在task1文件架中
test=pd.read_csv("../task1/test.tsv", delimiter="\t").drop(columns=["PhraseId","SentenceId"])
train=pd.read_csv("../task1/train.tsv", delimiter="\t").drop(columns=["PhraseId","SentenceId"])
train.info()

In [ ]:
class data(Dataset):
    def __init__(self) -> None:
        super(data,self).__init__()

    def __getitem__(self, index) -> None:
        return super().__getitem__(index)
    

    
        

In [7]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,dilation,A=False):
        super(CausalConv1d, self).__init__()
        self.kernel_size=kernel_size
        self.dilation=dilation
        self.A=A
        # 最优padding方式，可以根据是否是A层，来决定是否对当前的数据进行卷积
        self.padding=(kernel_size-1)*dilation+A*1
        self.conv1d=nn.Conv1d(in_channels,out_channels,kernel_size,\
                              stride=1,padding=0,dilation=dilation)
        # self.padding = (kernel_size - 1, 0)  # 在序列的左侧进行padding，右侧不进行padding
        # self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=0)

    def forward(self, x):
        x = torch.nn.functional.pad(x, (self.padding,0))  # 序列左侧padding
        conv_out=self.conv1d(x)
        if(self.A):
            return conv_out[:,:,:-1]
        else:
            return conv_out
        
        # x = self.conv(x)
        # return x[:, :, :-self.padding[0]]  # 去掉输出序列的右侧多余部分

# 创建输入张量，形状为 [batch_size, in_channels, sequence_length]
input = torch.randn(1, 3, 8)

# 创建因果卷积层
causal_conv = CausalConv1d(in_channels=3, out_channels=16, kernel_size=2,dilation=3)

# 进行因果卷积操作
output = causal_conv(input)

print(output.shape)

torch.Size([1, 16, 8])


In [ ]:
del train
del test